## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### Шаг 1. Изучим общую информацию. 

In [1]:
import pandas as pd
data = pd.read_csv('/datasets/data.csv') 
print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB
None


In [2]:
print(data.head())

   children  days_employed  dob_years education  education_id  \
0         1   -8437.673028         42    высшее             0   
1         1   -4024.803754         36   среднее             1   
2         0   -5623.422610         33   Среднее             1   
3         3   -4124.747207         32   среднее             1   
4         0  340266.072047         53   среднее             1   

      family_status  family_status_id gender income_type  debt   total_income  \
0   женат / замужем                 0      F   сотрудник     0  253875.639453   
1   женат / замужем                 0      F   сотрудник     0  112080.014102   
2   женат / замужем                 0      M   сотрудник     0  145885.952297   
3   женат / замужем                 0      M   сотрудник     0  267628.550329   
4  гражданский брак                 1      F   пенсионер     0  158616.077870   

                      purpose  
0               покупка жилья  
1     приобретение автомобиля  
2               покупка жи

### Вывод

Заметим, что количество пустых значений в day_employed и total_income одинаково. Займемся заполнением пустых значений. Заметим, что у пенсионера 100 лет стажа, это артефакт, с которым мы разберемся позже.

### Шаг 2. Предобработка данных

### Обработка пропусков

Посмотрим количество пропусков в ежемесячном доходе для каждого типа занятости:

In [3]:
print(data[data['total_income'].isnull()]['income_type'].value_counts())



сотрудник          1105
компаньон           508
пенсионер           413
госслужащий         147
предприниматель       1
Name: income_type, dtype: int64


Проверим теперь для общего трудового стажа:

In [4]:
print(data[data['days_employed'].isnull()]['income_type'].value_counts())

сотрудник          1105
компаньон           508
пенсионер           413
госслужащий         147
предприниматель       1
Name: income_type, dtype: int64


Заметим, что выводы одинаковые. Убедимся, что если пропущен стаж, то и ежемесячный доход тоже пропущен:

In [5]:
print(data[(data['total_income'].isnull()) & (data['days_employed'].isnull())]['income_type'].value_counts())

сотрудник          1105
компаньон           508
пенсионер           413
госслужащий         147
предприниматель       1
Name: income_type, dtype: int64


Такие ошибки могут быть обусловленны техническими сбоями.  
Заполнять пропуски будем с помощью медианы, так как мы хотим сохранить параметры распределения заполняемой переменной.  
Теперь заполним пропуски с помощью группировки по одной переменной:

In [6]:
print('Пропуски в стаже работы до заполнения:', data['days_employed'].isna().sum())
print('Пропуски в ежемесячном доходе до заполнения:', data['total_income'].isna().sum())

for inctype in data['income_type'].unique():
    #считаем медиану для определенного типа занятости
    median_days_employed = data.loc[data['income_type'] == inctype,'days_employed'].median()
    median_total_income = data.loc[data['income_type'] == inctype,'total_income'].median()
    print('Медиана стажа: {:.2f}'.format(median_days_employed), inctype, ', медиана зарплаты: {:.2f}'.format(median_total_income), inctype)
    #подставляем вместо пустых значений медиану    
    data.loc[(data['days_employed'].isna()) & (data['income_type'] == inctype),'days_employed'] = median_days_employed
    data.loc[(data['total_income'].isna()) & (data['income_type'] == inctype),'total_income'] = median_total_income

print('Пропуски в стаже работы после заполнения:',data['days_employed'].isna().sum())
print('Пропуски в ежемесячном доходе после заполнения:',data['total_income'].isna().sum())
print(data[data['days_employed'].isnull()]['income_type'].value_counts())

Пропуски в стаже работы до заполнения: 2174
Пропуски в ежемесячном доходе до заполнения: 2174
Медиана стажа: -1574.20 сотрудник , медиана зарплаты: 142594.40 сотрудник
Медиана стажа: 365213.31 пенсионер , медиана зарплаты: 118514.49 пенсионер
Медиана стажа: -1547.38 компаньон , медиана зарплаты: 172357.95 компаньон
Медиана стажа: -2689.37 госслужащий , медиана зарплаты: 150447.94 госслужащий
Медиана стажа: 366413.65 безработный , медиана зарплаты: 131339.75 безработный
Медиана стажа: -520.85 предприниматель , медиана зарплаты: 499163.14 предприниматель
Медиана стажа: -578.75 студент , медиана зарплаты: 98201.63 студент
Медиана стажа: -3296.76 в декрете , медиана зарплаты: 53829.13 в декрете
Пропуски в стаже работы после заполнения: 0
Пропуски в ежемесячном доходе после заполнения: 0
Series([], Name: income_type, dtype: int64)


### Вывод

Заметим значения, не отображающие действительность:  
1) Стаж работы отрицательный (возможно стаж считался разностью начальной даты и конечной С=Н-К)  
2) Медиана пенсионеров и безработных превышает 100 лет  (Скорее всего данные получались в других единицах измерения)

# Замена типа данных

Вещественные значения стажа и прибыли не несут информации и мешают визуальному восприятию, поэтому преобразуем данные из float64 в int64 и уберем отрицательные значения(возьмем по модулю) 

In [7]:
data['days_employed'] = data['days_employed'].astype(int) # используем метод astype для замены типа данных
data['total_income'] = data['total_income'].astype(int)
data['days_employed'] = data['days_employed'].apply(abs) #применяем к столбцу 'days_employed' метод abs(модуль)
data['children'] = data['children'].apply(abs)

Убедимся, что наши преобразование не нарушили медиану

In [8]:
for inctype in data['income_type'].unique():
    #считаем медиану для определенного типа занятости
    median_days_employed = data.loc[data['income_type']==inctype,'days_employed'].median()
    median_total_income = data.loc[data['income_type']==inctype,'total_income'].median()
    print('Медиана стажа: {:.0f}'.format(median_days_employed), inctype, ', медиана зарплаты: {:.0f}'.format(median_total_income), inctype)


Медиана стажа: 1574 сотрудник , медиана зарплаты: 142594 сотрудник
Медиана стажа: 365213 пенсионер , медиана зарплаты: 118514 пенсионер
Медиана стажа: 1547 компаньон , медиана зарплаты: 172357 компаньон
Медиана стажа: 2689 госслужащий , медиана зарплаты: 150447 госслужащий
Медиана стажа: 366413 безработный , медиана зарплаты: 131339 безработный
Медиана стажа: 520 предприниматель , медиана зарплаты: 499163 предприниматель
Медиана стажа: 578 студент , медиана зарплаты: 98201 студент
Медиана стажа: 3296 в декрете , медиана зарплаты: 53829 в декрете


### Вывод

Наши преобразования не повлияли на общую картину, и мы можем продолжить заниматься предобработкой

### Обработка дубликатов

In [9]:
print('Найдено дупликатов:',data.duplicated().sum())

Найдено дупликатов: 54


In [10]:
data=data.drop_duplicates().reset_index(drop=True)# используем метод drop_duplicates().reset_index(drop=True)
print('Найдено дупликатов после удаления:',data.duplicated().sum())

Найдено дупликатов после удаления: 0


### Вывод

Появление дупликатов может быть вызвано тем, что пользователь заполнил форму дважды, или отправка данных повторилась. Возможно, что некоторые строки повтаряются из-за заполнения пропусков, но это маленький процент, которым можно пренебречь.

### Лемматизация

In [11]:
print(data['education'].unique())

['высшее' 'среднее' 'Среднее' 'СРЕДНЕЕ' 'ВЫСШЕЕ' 'неоконченное высшее'
 'начальное' 'Высшее' 'НЕОКОНЧЕННОЕ ВЫСШЕЕ' 'Неоконченное высшее'
 'НАЧАЛЬНОЕ' 'Начальное' 'Ученая степень' 'УЧЕНАЯ СТЕПЕНЬ'
 'ученая степень']


Приведем все к нижнему регистру

In [12]:
data['education']=data['education'].str.lower()
print(data['education'].unique())

['высшее' 'среднее' 'неоконченное высшее' 'начальное' 'ученая степень']


In [13]:
# Получим наиболее популярные слова в столбце целей
from nltk.stem import SnowballStemmer
from pymystem3 import Mystem
import numpy as np
m = Mystem()
russian_stemmer=SnowballStemmer('russian')
data_puprose_unique=(data['purpose'].unique())
myString = ' '.join(data_puprose_unique)
myString=m.lemmatize(myString)
from collections import Counter
print(Counter(myString))


Counter({' ': 96, 'покупка': 10, 'недвижимость': 10, 'автомобиль': 9, 'образование': 9, 'жилье': 7, 'с': 5, 'операция': 4, 'на': 4, 'свой': 4, 'свадьба': 3, 'строительство': 3, 'получение': 3, 'высокий': 3, 'дополнительный': 2, 'для': 2, 'коммерческий': 2, 'жилой': 2, 'подержать': 2, 'заниматься': 2, 'сделка': 2, 'приобретение': 1, 'сыграть': 1, 'проведение': 1, 'семья': 1, 'собственный': 1, 'со': 1, 'профильный': 1, 'сдача': 1, 'ремонт': 1, '\n': 1})


Проверим, появились ли новые дупликаты.

In [14]:
print('Найдено дупликатов:',data.duplicated().sum())

Найдено дупликатов: 17


Появились новые дупликаты, уберем их.

In [15]:
data=data.drop_duplicates().reset_index(drop=True)# используем метод drop_duplicates().reset_index(drop=True)
print('Найдено дупликатов после удаления:',data.duplicated().sum())

Найдено дупликатов после удаления: 0


### Вывод

Мы привели все к нижнему регистру и выявили самые популярные слова в столбце 'purpose'. Его можно будет разделить на более удобные группы: 'недвижимость', 'автомобиль', 'образование', 'свадьба'

### Категоризация данных

В процессе предобработки можно было заметить, что можно выделить следующие словари:  
1) образование  
2) семейное положение  
3) цель получения кредита



In [16]:
data_education_dict=data.groupby(['education_id','education']).agg({'total_income':'count'})
print(data_education_dict,'\n')
data_family_dict=data.groupby(['family_status_id','family_status']).agg({'total_income':'count'})
print(data_family_dict,'\n')

from nltk.stem import SnowballStemmer
from pymystem3 import Mystem
import numpy as np
m = Mystem()
russian_stemmer=SnowballStemmer('russian')
# Функция, получающая строку данных и возвращающая категорию цели кредита.
def lemm(row):
    purp = row['purpose']
    lemmas = m.lemmatize(purp)
    if 'жилье' in lemmas:
        return('недвижимость')
    elif 'недвижимость' in lemmas:
        return('недвижимость')
    elif 'автомобиль' in lemmas:
        return 'автомобиль'
    elif 'свадьба' in lemmas: 
        return 'свадьба'
    elif 'образование' in lemmas: 
        return 'образование'
    
data['purpose_type']=data.apply(lemm,axis=1)
data_purpose_dict=data.groupby(['purpose_type','purpose']).agg({'total_income':'count'})
print(data_purpose_dict)

                                  total_income
education_id education                        
0            высшее                       5250
1            среднее                     15172
2            неоконченное высшее           744
3            начальное                     282
4            ученая степень                  6 

                                        total_income
family_status_id family_status                      
0                женат / замужем               12339
1                гражданский брак               4151
2                вдовец / вдова                  959
3                в разводе                      1195
4                Не женат / не замужем          2810 

                                                     total_income
purpose_type purpose                                             
автомобиль   автомобили                                       478
             автомобиль                                       494
             на покупку автомоби

### Вывод

Больше всего кредиты берут люди в браке со средним образованием.

### Шаг 3. Ответы на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [17]:
report_children=data.groupby('children').agg({'debt':['mean','count']})
report_children.columns=['%невозврата','количество заемщиков']
report_children.style.format({'%невозврата':'{:.2%}'})


,%невозврата,количество заемщиков
children,,
0,7.54%,14091
1,9.17%,4855
2,9.45%,2052
3,8.18%,330
4,9.76%,41
5,0.00%,9
20,10.53%,76


20 детей выглядит неправдоподобно. Этим артефактом обладают 76 человек из 20 тыс, следует их убрать.

In [18]:
data=data.loc[data['children']!=20]
report_children=data.groupby('children').agg({'debt':['mean','count']})
report_children.columns=['%невозврата','количество заемщиков']
report_children.style.format({'%невозврата':'{:.2%}'})

,%невозврата,количество заемщиков
children,,
0,7.54%,14091
1,9.17%,4855
2,9.45%,2052
3,8.18%,330
4,9.76%,41
5,0.00%,9


### Вывод

Процент невозврата находится в диапазоне от 7 до 10 (5 детей не учитываем тк малое количество заемщиков). Заметим, что бездетные имеют наименьший процент невозврата.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [19]:
report_family_status= data.pivot_table(index=['family_status'],values=['debt'],aggfunc=['mean','count'])
report_family_status.columns=['%невозврата','количество заемщиков']
report_family_status=report_family_status.sort_values(by='количество заемщиков',ascending=False)
report_family_status.style.format({'%невозврата':'{:.2%}'})

,%невозврата,количество заемщиков
family_status,,
женат / замужем,7.55%,12290
гражданский брак,9.30%,4139
Не женат / не замужем,9.75%,2801
в разводе,7.04%,1193
вдовец / вдова,6.60%,955


### Вывод

Люди, не вступавшие в брак, относятся к возвращению кредита халатнее. Вдовец / вдова имеют наименьший процент невозврата.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [46]:
income_mean=data['total_income'].mean()
income_median=data['total_income'].median()
income_max=data['total_income'].max()
print('Самая большая зарплата',income_max)
print('Среднее значение зарплат: ',int(income_mean))
print('Медиана зарплат: ',int(income_median))
data['income_level']=data['total_income']
data.loc[data['total_income']<=income_median/2,'income_level']=('<'+str(int(income_median/2))+'тыс')
data.loc[(data['total_income']>income_median/2) & (data['total_income']<=income_median),'income_level']=(str(int(income_median/2))+'<'+str(int(income_median))+'тыс')
data.loc[(data['total_income']>income_median),'income_level']=('>'+str(int(income_median))+'тыс')

#data.loc[(data['days_employed'].isna()) & (data['income_type'] == inctype),'days_employed'] = median_days_employed

report_income_level=data.groupby('income_level').agg({'debt':['mean','count']})
report_income_level.columns=['%невозврата','количество заемщиков']
report_income_level.style.format({'%невозврата':'{:.2%}'})

Самая большая зарплата 2265604
Среднее значение зарплат:  165316
Медиана зарплат:  142594


,%невозврата,количество заемщиков
income_level,,
71297<142594тыс,8.64%,9232
<71297тыс,6.93%,1574
>142594тыс,7.81%,10572


### Вывод

Удивительно, но самыми ответственными являются люди, с уровнем дохода <71 тыс рублей. После них идут люди с зарплатой >142 тыс, которым легче возвращать кредит. 	

- Как разные цели кредита влияют на его возврат в срок?

In [21]:
report_purpose_type=data.groupby('purpose_type').agg({'debt':['mean','count']})
report_purpose_type.columns=['%невозврата','количество заемщиков']
report_purpose_type.style.format({'%невозврата':'{:.2%}'})

,%невозврата,количество заемщиков
purpose_type,,
автомобиль,9.35%,4290
недвижимость,7.24%,10775
образование,9.23%,3998
свадьба,7.90%,2315


### Вывод

Заемщики, берущие кредит ради недвижимости, подходят к возвращению долга ответственнее. Автомоюиль является наиболее эмоциональой покупкой, поэтому и возвращают его неохотнее всего.

### Шаг 4. Общий вывод

In [49]:
report_purpose_type.style.format({'%невозврата':'{:.2%}'})


,%невозврата,количество заемщиков
purpose_type,,
автомобиль,9.35%,4290
недвижимость,7.24%,10775
образование,9.23%,3998
свадьба,7.90%,2315


In [50]:
report_income_level.style.format({'%невозврата':'{:.2%}'})

,%невозврата,количество заемщиков
income_level,,
71297<142594тыс,8.64%,9232
<71297тыс,6.93%,1574
>142594тыс,7.81%,10572


In [51]:
report_family_status.style.format({'%невозврата':'{:.2%}'})

,%невозврата,количество заемщиков
family_status,,
женат / замужем,7.55%,12290
гражданский брак,9.30%,4139
Не женат / не замужем,9.75%,2801
в разводе,7.04%,1193
вдовец / вдова,6.60%,955


Самыми ответственными для возвращения кредита являются бездетные пары(как раз на старте отношений зп <71 тыс), берущие кредит для покупки новой недвижимости. Самыми рискованными клиентами являются люди, находящиеся не в официальном браке, с зарплатой 71 < 142 тыс, которые берут кредит на эмоциональные покупки(автомобиль).